<a href="https://colab.research.google.com/github/ParthGadekar0631/Algorithmic-Trading-System/blob/main/Algorithmic_Trading_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
!pip install quandl # Install the quandl package
import quandl as q # Import the package after installation

In [ ]:
# set the API key
q.ApiConfig.api_key = "<API key>”

#send a get request to query Microsoft's end of day stock prices from 1st #Jan, 2010 to 1st Jan, 2019
msft_data = q.get("EOD/MSFT", start_date="2010-01-01", end_date="2019-01-01")

# look at the first 5 rows of the dataframe
msft_data.head()

msft_data.describe()

msft_data.resample('M').mean()

In [2]:
# Import numpy package
import numpy as np

# assign `Adj Close` to `daily_close`
daily_close = msft_data[['Adj_Close']]

# returns as fractional change
daily_return = daily_close.pct_change()

# replacing NA values with 0
daily_return.fillna(0, inplace=True)

print(daily_return)

NameError: name 'msft_data' is not defined

In [ ]:
mdata = msft_data.resample('M').apply(lambda x: x[-1])
monthly_return = mdata.pct_change()

In [ ]:
Lambda: expression

In [ ]:
# assigning adjusted closing prices to
adj_pricesadj_price = msft_data['Adj_Close']

# calculate the moving average
mav = adj_price.rolling(window=50).mean()

# print the result
print(mav[-10:])

In [ ]:
# import the matplotlib package to see the plot

import matplotlib.pyplot as plt
adj_price.plot()
mav.plot()

# step1: initialize the short and long lookback periods
short_lb = 50long_lb = 120

# step2: initialize a new DataFrame called signal_df with a signal column
signal_df = pd.DataFrame(index=msft_data.index)signal_df['signal'] = 0.0

# step3: create a short simple moving average over the short lookback period
signal_df['short_mav'] = msft_data['Adj_Close'].rolling(window = short_lb, min_periods=1, center=False).mean()

# step4: create long simple moving average over the long lookback period
signal_df['long_mav'] = msft_data['Adj_Close'].rolling(window=long_lb, min_periods=1, center=False).mean()

# step5: generate the signals based on the conditional statement
signal_df['signal'][short_lb:] = np.where(signal_df['short_mav'][short_lb:] > signal_df['long_mav'][short_lb:], 1.0, 0.0)

# step6: create the trading orders based on the positions column
signal_df['positions'] = signal_df['signal'].diff()signal_df[signal_df['positions'] == -1.0]

# initialize the plot using plt
fig = plt.figure()

# Add a subplot and label for y-axis
plt1 = fig.add_subplot(111,  ylabel='Price in $')

msft_data['Adj_Close'].plot(ax=plt1, color='r', lw=2.)

# plot the short and long lookback moving averages
signal_df[['short_mav', 'long_mav']].plot(ax=plt1, lw=2., figsize=(12,8))

# plotting the sell signals
plt1.plot(signal_df.loc[signal_df.positions == -1.0].index, signal_df.short_mav[signal_df.positions == -1.0],'v', markersize=10, color='k')

# plotting the buy signals
plt1.plot(signal_df.loc[signal_df.positions == 1.0].index, signal_df.short_mav[signal_df.positions == 1.0],'^', markersize=10, color='m')

# Show the plot
plt.show()